In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
import os
import dotenv
dotenv.load_dotenv()

True

In [4]:
from pydantic import BaseModel
from pathlib import Path

class ProcessConfig(BaseModel):
    connection_string: str
    table_name: str
    container_name: str

    class Config:
        frozen = True  # To make the model immutable like the frozen dataclass


In [5]:
from Ingest.constants import *
from Ingest.utils.common import read_yaml, read_table_to_dataframe, upload_dataframe_to_blob

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        params_filepath = PARAMS_FILE_PATH):

        self.params = read_yaml(params_filepath)


    
    def get_process_config(self) -> ProcessConfig:

        config = self.params


        process_config = ProcessConfig(

            table_name=config.table_name,

            connection_string=os.getenv('CONNECTION_STRING'),
            
            container_name=config.container_name
        )

        return process_config

In [7]:
class Process:
    def __init__(self, config: ProcessConfig):
        self.config = config
    

    def start_process(self):
        df = read_table_to_dataframe(self.config.connection_string, self.config.table_name)
        upload_dataframe_to_blob(df, self.config.container_name, 'test.csv', self.config.connection_string)

In [8]:
config = ConfigurationManager()
process_config = config.get_process_config()
processing = Process(config=process_config)
process_data = processing.start_process()

[2024-08-20 17:44:23,684: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-20 17:44:23,687: INFO: _universal: Request URL: 'https://csm1klagsto902.table.core.windows.net/testyahoo()'
Request method: 'GET'
Request headers:
    'x-ms-version': 'REDACTED'
    'DataServiceVersion': 'REDACTED'
    'Accept': 'application/json;odata=minimalmetadata'
    'x-ms-client-request-id': '12d302dc-5f0b-11ef-98e2-000d3abb4ec5'
    'x-ms-date': 'REDACTED'
    'Date': 'Tue, 20 Aug 2024 15:44:23 GMT'
    'User-Agent': 'azsdk-python-data-tables/12.5.0 Python/3.9.16 (Windows-10-10.0.17763-SP0)'
    'Authorization': 'REDACTED'
No body was attached to the request]
[2024-08-20 17:44:23,892: INFO: _universal: Response status: 200
Response headers:
    'Cache-Control': 'no-cache'
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/json;odata=minimalmetadata;streaming=true;charset=utf-8'
    'Server': 'Windows-Azure-Table/1.0 Microsoft-HTTPAPI/2.0'
    'x-ms-request-id': '502b227